In [ ]:
!pip install transformers accelerate datasets torch torchvision peft pillow

In [2]:
from transformers import pipeline
model_name = "Qwen/Qwen2.5-3B-Instruct"
ask_llm = pipeline(
    model = model_name,
    device ="cuda"
)
print(ask_llm('who is Ferdousi')[0]['generated_text'])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda


who is Ferdousi? Ferdowsi, born in 940 AD, was an Iranian poet and writer of Persian epic poetry. He is best known for his monumental work, the Shahnameh (Book of Kings), which chronicles the mythical and historical past of Iran from the creation of the world to the Islamic conquest. The Shahnameh is considered one of the greatest works of Persian literature and is still widely read and admired today. Ferdowsi was a member of the Zoroastrian clergy and spent much of his life working on the Shahnameh, which he completed in 1010 AD. He is regarded as the national poet of Iran and his works continue to be studied and celebrated in Iran and around the world.


In [5]:
from datasets import load_dataset
raw_data = load_dataset('json',
                      data_files='/content/child_psychology_QA.json',
                      encoding='utf-8-sig')  # This handles the BOM
print(raw_data)

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 48
    })
})


In [6]:
x = raw_data['train'][2]
print(x)

{'prompt': 'بهترین راه آموزش نظم به کودک 5 ساله چیست؟', 'completion': 'تعیین قوانین واضح، اجرای پیوسته آن\u200cها و استفاده از پاداش مثبت برای رفتارهای مطلوب بهترین نتیجه را می\u200cدهد.'}


In [8]:
#tokenizer

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess(sample):
    sample = sample["prompt"] + "\n" + sample['completion']
    tokenized = tokenizer(
        sample ,
        max_length= 256,
        truncation = True,
        padding = "max_length",
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

data = raw_data.map(preprocess)

In [7]:
print(data["train"][2])

{'prompt': 'بهترین راه آموزش نظم به کودک 5 ساله چیست؟', 'completion': 'تعیین قوانین واضح، اجرای پیوسته آن\u200cها و استفاده از پاداش مثبت برای رفتارهای مطلوب بهترین نتیجه را می\u200cدهد.', 'input_ids': [126751, 124015, 82255, 53710, 124642, 125108, 10176, 124412, 32790, 50243, 125362, 81768, 45593, 69423, 32311, 220, 20, 59842, 31382, 16157, 220, 144751, 14391, 46586, 128332, 198, 124464, 14391, 82255, 77703, 126277, 82255, 136315, 68785, 12961, 124328, 46072, 87417, 14391, 12653, 46586, 16157, 125108, 11798, 89364, 124006, 37524, 93153, 135971, 16157, 12961, 39697, 87417, 65398, 126409, 23364, 135445, 129920, 46072, 53710, 128583, 35038, 16157, 46072, 23364, 124525, 124495, 81768, 124015, 82255, 50243, 14293, 14391, 124649, 53710, 5703, 23364, 14391, 89364, 91335, 13325, 13, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 15164

In [2]:
from peft import LoraConfig, get_peft_model, TaskType
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
import torch

!pip install bitsandbytes accelerate

# Configure 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# Load model with 4-bit quantization
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

# Configure LoRA
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    target_modules=["q_proj", "k_proj", "v_proj"],  # For Qwen2
    r=8,  # LoRA rank
    lora_alpha=32,
    lora_dropout=0.05
)

# Apply LoRA
model = get_peft_model(model, lora_config)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
from transformers import TrainingArguments , Trainer

training_args = TrainingArguments(
    num_train_epochs = 10,
    learning_rate = 0.001,
    logging_steps = 50,
    per_device_train_batch_size=2,
)

trainer = Trainer(
    model = model ,
    args = training_args,
    train_dataset = data['train']
)
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: fa-kashany (fa-kashany-uae) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
50,0.903600
100,0.254700
150,0.126200
200,0.054700


TrainOutput(global_step=240, training_loss=0.2841114059090614, metrics={'train_runtime': 213.2646, 'train_samples_per_second': 2.251, 'train_steps_per_second': 1.125, 'total_flos': 2047633375887360.0, 'train_loss': 0.2841114059090614, 'epoch': 10.0})

In [20]:
#Clear Cache
import torch
torch.cuda.empty_cache()

In [11]:
trainer.save_model("./child_psychology")
tokenizer.save_pretrained("./child_psychology")

('./child_psychology/tokenizer_config.json',
 './child_psychology/special_tokens_map.json',
 './child_psychology/chat_template.jinja',
 './child_psychology/vocab.json',
 './child_psychology/merges.txt',
 './child_psychology/added_tokens.json',
 './child_psychology/tokenizer.json')

In [ ]:
#this test failed with RAM crash!
from transformers import AutoTokenizer , AutoModelForCausalLM
from peft import PeftModel ,PeftConfig

path = "./child_psychology"
config = PeftConfig.from_pretrained(path)

base = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path ,trust_remote_code=True)
model = PeftModel.from_pretrained(base,path)

tokenizer = AutoTokenizer.from_pretrained(path ,trust_remote_code=True)

inputs = tokenizer ('درمورد دروغ گویی کودکان بگو' , return_tensors="pt").to(model.device)

output = model.generate(
    input_ids = inputs["input_ids"] ,
    attention_mask = inputs["attention_mask"]

)

print(tokenizer.decode(output[0]))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [19]:
#test
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

# Configure 4-bit loading
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# Load base model with quantization
base = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map="auto"
)

# Load PEFT adapter
model = PeftModel.from_pretrained(
    base,
    path,
    device_map="auto"
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True)

# Move inputs to the same device as model
inputs = tokenizer('درمورد دروغ گویی کودکان بگو', return_tensors="pt").to(model.device)

# Generate with optimized settings
with torch.no_grad():  # Disable gradient calculation to save memory
    output = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=100,
        do_sample=True,
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id  # Important for some models
    )

print(tokenizer.decode(output[0], skip_special_tokens=True))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

درمورد دروغ گویی کودکان بگویی می‌شود
بازی‌های وسیله‌ای، حرف‌بازی‌های حرکتی و اجرا دستورالعمل‌های خواهر/
